# Plug2Drive Automated Report

## Table of Contents
1. [Retrieve All Matching Folders](#Retrieve-All-Matching-Folders)
2. [Find and Extract Data from an Excel File](#Find-and-Extract-Data-from-an-Excel-File)

  - [Final dictionary](#Final-dictionary)
  - [Data collected from "Info" Sheet](#Data-collected-from-"Info"-Sheet)
  - [Data collected from "Équilibrage" Sheet (site and logement only)](#Data-collected-from-"Équilibrage"-Sheet-(site-and-logement-only))
  - [Data collected from "Équilibrage" Sheet (table element only)](#Data-collected-from-"Équilibrage"-Sheet-(table-element-only))
  - [Data collected from "Températures" Sheet](#Data-collected-from-"Températures"-Sheet)
3. [Locate and Extract image from Images/subfolder](#Locate-and-Extract-image-from-Images/subfolder)
4. [Create PDF page linked to each site](#Create-PDF-page-linked-to-each-site)
  - [Header/Footer + Custom Page Number](#Header/Footer-+-Custom-Page-Number)
  - [Page 1: Title + Info Table](#Page-1:-Title-+-Info-Table)
  - [Page 5: Installation et Réseaux](#Page-5:-Installation-et-Réseaux)
  - [Page 7: Tableau Relevé Equilibrage](#Page-7:-Tableau-Relevé-Equilibrage)
  - [Page 8: Tableau Enregistrement Températures](#Page-8:-Tableau-Enregistrement-Températures)
  - [Page 10: Vannes Photos](#Page-10:-Vannes-Photos)
  
  
5. [Combining the Output Report with the Template](#Combining-the-Output-Report-with-the-Template)
6. [Execute All Functions Together](#Execute-All-Functions-Together)

In [45]:
import os
import re
from typing import List, Dict, Optional
from pathlib import Path
import pandas as pd
from fpdf import FPDF
import textwrap
from PyPDF2 import PdfReader, PdfWriter
from PIL import Image
from datetime import datetime
from fpdf.enums import XPos, YPos

# Retrieve All Matching Folders

In [2]:
def get_site_folder() -> List[Path]:
    """
    Function objective: Find and return folders matching the specified pattern (here full_path_pattern)
    as a list.
    
    :return: Path that matches full_path_pattern variable
    :rtype: list 
    
    :Example:
    >>> get_site_folder()
    [PosixPath('/Users/.../RAPPORTS_EQ-XX/TX/RXXX_Site XXXXX'), 
    PosixPath('/Users/.../RAPPORTS_EQ-XX/TX/RXXX_Site XXXXX'),
    ...,
    PosixPath('/Users/.../RAPPORTS_EQ-XX/TX/RXXX_Site XXXXX')
    ]
    """
    site_folder_path = []
    full_path_pattern = r"([A-Za-z]:\\|/)?RAPPORTS_EQ-\d{2}(\\|/)T[1-4](\\|/)R\d{3}_Site [A-Za-zÀ-ÿ\s\-\d]+$"
    base_dir = Path.home()
    
    for path in base_dir.rglob("*"):
        if path.is_dir() and re.search(full_path_pattern, str(path)):
            site_folder_path.append(path)
            
    if not site_folder_path:
        print("No matching folders were found.")
        
    return site_folder_path

# TEST
#get_site_folder()

# Find and Extract Data from an Excel File

### Final dictonary

In [3]:
def extract_excel_data(excel_file: Path) -> Optional[Dict]:
    """
    Function objective:
    Combine dictionaries got from the following funtions:
        - extract_info_sheet()
        - extract_equilibrage_site_logement()
        - extract_equilibrage_table_data()
        - extract_temperatures_data()
    
    :param excel_file: path of folder's Excel file
    :type excel_file: Path
    :return: data retrieved from "Info", "Équilirage" and "Températures" sheets
    :rtype: dict
    
    :Example:
    >>> extract_excel("/Users/.../RAPPORTS_EQ-XX/TX/RXXX_Site XXXXX/donnees_clientsxxx.xlsx")
    {'Info': {'FICHE': 'XXXX',
      'BÉNÉFICIAIRE': 'XXXX',
      'PRESTATAIRE': 'XXXX',
      'INTERVENANT': 'XXXX',
      'TECHNICIEN': 'XXXX',
      "DATE D'INTERVENTION": 'XXXXXX'},
     'Équilibrage_Site_Logement': {'Site_Info': 'XXXXXX',
      'Logement_Info': 'Nombre de logement : XXXX'},
     'Équilibrage_Table_Data': {'Repérage vanne': [XXXXXX],
      'Localisation': ['XXXXX],
      'Référence': [XXXXXX],
      'Marque': [XXXXX],
      'DN': [XXXX],
      'Débit Théorique (l/h)': [XXXXX],
      'Débit mesuré litres/h': [XXXXX],
      'Réglage (Tours)': [XXXXXX],
      'Conf': [XXXXXX]},
     'Températures': {'Température extérieure': [XXXXX],
      'Date': [XXXXX],
      'Heure': [XXXXX],
      'Localisation': [XXXXXX],
      'Température': [XXXXXX],
      'Écarts': [XXXXXXX]}}
    """    
    try:
        donnees_clients_data = {}

        # Extract data from each sheet using helper functions
        donnees_clients_data['Info'] = extract_info_sheet(excel_file)
        donnees_clients_data['Équilibrage_Site_Logement'] = extract_equilibrage_site_logement(excel_file)
        donnees_clients_data['Équilibrage_Table_Data'] = extract_equilibrage_table_data(excel_file)
        donnees_clients_data["Températures"] = extract_temperatures_data(excel_file)

        return donnees_clients_data
    except Exception as e:
        print(f"Failed to read Excel file {excel_file}: {e}")
        return None
    

# TEST
#extract_excel_data('/Users/.../Data Analysis/Automatisation-Rapport_Gabriella/RAPPORTS_EQ-34/T4/R991_Site K/donnees_clients100.xlsx')

### Data collected from "Info" Sheet

In [4]:
def extract_info_sheet(excel_file: Path) -> Optional[Dict]:
    """
    The purpose of this function is to extract data from the "Info" sheet of the Excel file, 
    and structure it into a dictionary.
    It also modify date format of data located inside "DATE D'INTERVENTION" column.
    If the Excel file has no "Info" sheet, the function raises a Value Error.
    
    :param excel_file: Path of the Excel file
    :type excel_file: Path
    :return: Data retrieved from "Info" sheet.
    :rtype: dict
    :raises ValueError: If the "Info" sheet is not found.
    
    :Example:
    >>> extract_info_sheet("/Users/.../RAPPORTS_EQ-XX/TX/RXXX_Site XXXXX/donnees_clientsxxx.xlsx")
    {'FICHE': 'XXXX',
     'BÉNÉFICIAIRE': 'XXXX',
     'PRESTATAIRE': 'XXXX',
     'INTERVENANT': 'XXXXX',
     'TECHNICIEN': 'XXXX',
     "DATE D'INTERVENTION": 'DD/MM/YYYY'}
    """
    
    try:
        info_sheet = pd.read_excel(excel_file, sheet_name="Info")
    except ValueError as e:
        raise ValueError(f"Sheet 'Info' not found in {excel_file}")
    
    info_sheet_dict = info_sheet.to_dict(orient="records")[0]
    
    if "DATE D'INTERVENTION" in info_sheet_dict and isinstance(info_sheet_dict["DATE D'INTERVENTION"], pd.Timestamp):
        info_sheet_dict["DATE D'INTERVENTION"] = info_sheet_dict["DATE D'INTERVENTION"].strftime('%d/%m/%Y')
        
    return info_sheet_dict

# TEST
#extract_info_sheet('/Users/.../Data Analysis/Automatisation-Rapport_Gabriella/RAPPORTS_EQ-34/T4/R991_Site K/donnees_clients100.xlsx')

### Data collected from "Équilibrage" Sheet (site and logement only)

In [6]:
def extract_equilibrage_site_logement(excel_file: Path) -> Optional[Dict[str, str]]:
    """
    The purpose of this function is to extract data from the "Equilibrage" sheet of the Excel file, 
    and structure it into a dictionary.
    The dictionary will contain two keys: "Site_Info" and "Logement_Info".
    Their corresponding values will be the site's address name and the number of houses, respectively.
    If the Excel file has no "Équilibrage" sheet, the function raises a Value Error.
    
    :param excel_file: Path of the Excel file
    :type excel_file: Path
    :return: Data retrieved from "Équilibrage" sheet.
    :rtype: dict
    :raises ValueError: If the "Équilibrage" sheet is not found.
    
    :Example:
    >>> extract_info_sheet("/Users/.../RAPPORTS_EQ-XX/TX/RXXX_Site XXXXX/donnees_clientsxxx.xlsx")
    {'Site_Info': 'XXXXXXXXXXXXXX',
     'Logement_Info': 'XXXXXXXXXX'
     }
    """
    
    try:
        equilabrage_sheet1 = pd.read_excel(excel_file, sheet_name="Équilibrage")
    except ValueError as e:
        raise ValueError(f"Sheet 'Équilibrage' not found in {excel_file}")
        
    #site_logemt_do_client_equi = equilabrage_sheet1.iloc[1]  #do_client_equi1.iloc[1]
    
    site_logement_equi_sheet1 = equilabrage_sheet1.iloc[1]
    site_info = site_logement_equi_sheet1.iloc[0]
    logement_info = site_logement_equi_sheet1.iloc[6]
    
    equi_sheet1_dict = {"Site_Info": site_info,
        "Logement_Info": logement_info
                       }
        
    return equi_sheet1_dict

# TEST
#extract_equilibrage_site_logement('/Users/.../Data Analysis/Automatisation-Rapport_Gabriella/RAPPORTS_EQ-34/T4/R991_Site K/donnees_clients100.xlsx')

### Data collected from "Équilibrage" Sheet (table element only)

In [7]:
def extract_equilibrage_table_data(excel_file: Path) -> Optional[Dict[str, List[str]]]:
    """
    The purpose of this fonction is to extract data from the "Équilibrage" sheet of the Excel file found in each site folder
    and structure it into a dictionary.
    If the Excel file has no "Équilibrage" sheet, the function raises a Value Error.
    
    :param excel_file: Path of the Excel file
    :type excel_file: Path
    :return: Data retrieved from "Info" sheet.
    :rtype: dict
    :raises ValueError: If the "Équilibrage" sheet is not found.
    
    :Example:
    >>> extract_equilibrage_table_data("/Users/.../RAPPORTS_EQ-XX/TX/RXXX_Site XXXXX/donnees_clientsxxx.xlsx")
    {'Repérage vanne': [X, X, X, X, X],
     'Localisation': ['XXXXX', 'XXXXX',...,'XXXXX'],
     'Référence': ['XXXXX', 'XXXXX',...,'XXXXX'],
     'Marque': ['XXXXX', 'XXXXX',...,'XXXXX'],
     'DN': [XX, XX,...,XX],
     'Débit Théorique (l/h)': [XXX, XX, XXX,...,XXXX],
     'Débit mesuré litres/h': [XX, XX,...,XX],
     'Réglage (Tours)': [XX, XX,...,XX,]
     'Conf': ['X', 'X',...,'X']}       
     """
    
    try:
        equilibrage_sheet2 = pd.read_excel(excel_file, sheet_name="Équilibrage", skiprows=4)
    except ValueError as e:
        raise ValueError(f"Sheet 'Équilibrage' not found in {excel_file}")
        
    equi_sheet2_dict = equilibrage_sheet2.to_dict(orient='list')
        
    return equi_sheet2_dict         #do_client_equi2.to_dict(orient='list')

# TEST
#extract_equilibrage_table_data('/Users/.../Automatisation-Rapport_Gabriella/RAPPORTS_EQ-34/T4/R991_Site K/donnees_clients100.xlsx')

### Data collected from "Températures" Sheet

In [8]:
def extract_temperatures_data(excel_file: Path) -> Optional[Dict[str, List[str]]]:
    """
    Function objective:
    Extract data from the "Températures" sheet of the Excel file found in each site folder
    and structure it into a dictionary.
    If the Excel file has no "Températures" sheet, the function raises a Value Error.
    
    :param excel_file: Path of the Excel file
    :type excel_file: Path
    :return: Data retrieved from "Info" sheet.
    :rtype: dict
    :raises ValueError: If the "Températures" sheet is not found.
    
    :Example:
    >>> extract_temperatures_data("/Users/.../RAPPORTS_EQ-XX/TX/RXXX_Site XXXXX/donnees_clientsxxx.xlsx")
    {'Température extérieure': [XX, XX,..., XX],
    'Date': ['DD/MM/YYY',
    'DD/MM/YYY',...,
    'DD/MM/YYY'],
    'Heure': ['XXHXX', 'XXHXX', 'XXHXX',..., 'XXHXX'],
    'Localisation': ['XXXX', 'XXXXX',..., 'XXXX'],
    'Température': [XX, XX, XX,..., XX],
    'Écarts': [XX, XX,..., XX]}
    """
    
    try:
        temperatures_sheet = pd.read_excel(excel_file, sheet_name="Températures", skiprows=4)
    except ValueError as e:
        raise ValueError(f"Sheet 'Températures' not found in {excel_file}")
        
    temperatures_sheet.drop(["Unnamed: 6", "Unnamed: 7"], axis=1, inplace=True, errors='ignore')
    temperatures_sheet.dropna(how='all', inplace=True)
    temperatures_sheet["Date"] = temperatures_sheet["Date"].dt.strftime('%d/%m/%Y')
    temps_sheet_dict = temperatures_sheet.to_dict(orient='list')
    
    return temps_sheet_dict #temperatures_sheet.to_dict(orient='list')

# TEST
#extract_temperatures_data('/Users/.../Automatisation-Rapport_Gabriella/RAPPORTS_EQ-34/T4/R991_Site K/donnees_clients100.xlsx')

# Locate and Extract image from Images/subfolder

In [20]:
def extract_image(folder: Path) -> Optional[Dict[str, List[str]]]:
    """    
    Extract image paths from the 'Images' folder and its subfolders.
    
    This function first checks for the presence of the 'Images' directory inside the given folder.  
    If the directory does not exist, it raises a ValueError.  
    If the directory is found, the function checks for specific subfolders ("Chaufferie", "Façade", 
    "Pompes", "Vannes"). If a required subfolder is missing, it raises a ValueError.  
    For existing subfolders, the function searches for image files in JPEG and PNG formats  
    and structures the results into a dictionary.
    
    :param folder: Path of the folder
    :type folder: Path
    :return: Photos path of each subfolders
    :rtype: dict
    :raiseValueError: If the following element not found:
        - "Images"
        - "Images/Chaufferie", "Images/Façade", "Images/Pompes", "Images/Vannes"
        - JPEG or PNG format photos
        
    :Example:
    >>> extract_image(Path('/Users/.../RAPPORTS_EQ-XX/TX/RXXX_Site XXXXX'))
    {'Chaufferie': ['/Users/.../RAPPORTS_EQ-XX/TX/RXXX_Site XXXXX/Images/Chaufferie/XXXX.jpg'],
     'Façade': ['/Users/.../RAPPORTS_EQ-XX/TX/RXXX_Site XXXXX/Images/Façade/XXXXXX.jpg'],
     'Pompes': ['/Users/.../RAPPORTS_EQ-XX/TX/RXXX_Site XXXXX/Images/Pompes/XXXXXX.jpg',
      '/Users/.../RAPPORTS_EQ-XX/TX/RXXX_Site XXXXX/Images/Pompes/XXXXX.jpg'],
     'Vannes': ['/Users/.../RAPPORTS_EQ-XX/TX/RXXX_Site XXXXX/Images/Vannes/XXXXX.jpg',
      '/Users/.../RAPPORTS_EQ-XX/TX/RXXX_Site XXXXX/Images/Vannes/XXXXXX.jpg',
      '/Users/.../RAPPORTS_EQ-XX/TX/RXXX_Site XXXXX/Images/Vannes/IMG_3076.jpg']
    }
    """

    images_folder = folder.joinpath("Images")
    image_subfolders = ["Chaufferie", "Façade", "Pompes", "Vannes"]
    image_paths = {}
    

    if not images_folder.exists() or not images_folder.is_dir():
        raise ValueError(f"Error: The 'Images' directory is missing in {folder}")

    for subfolder in image_subfolders:
        subfolder_path = images_folder.joinpath(subfolder)

        if not subfolder_path.exists() or not subfolder_path.is_dir():
            raise ValueError(f"Error: The required subfolder '{subfolder}' is missing inside '{images_folder}'")

        photos = list(subfolder_path.glob("*.jpg")) + list(subfolder_path.glob("*.png"))

        if not photos:
            raise ValueError(f"Error: The subfolder '{subfolder}' in '{images_folder}' contains no images.")

        image_paths[subfolder] = [str(photo) for photo in photos]

    return image_paths if image_paths else None


# TEST
#extract_image(Path('/Users/.../Automatisation-Rapport_Gabriella/RAPPORTS_EQ-34/T4/R991_Site K'))

# Create PDF page linked to each site

## Header/Footer + Custom Page Number 

In [10]:
class CustomPDF(FPDF):
    
    def __init__(self, orientation="P", unit="mm", format="A4", total_pages=12):
        """
        This is a special method that allows us to modify the footer of 
        the custom PDF page we generate in the script, so that it matches 
        the page footer merged from the template in the final report
        
        :param orientation: Pdf sheet orientation
        :type orientation: str
        :param unit: Unit of measure use for the pdf
        :type unit: str
        :param format: Pdf sheet format
        :type format: str
        :param total_pages: PDF total pages
        :type total_pages: int
        """
        super().__init__(orientation, unit, format)
        self.total_pages = total_pages
        self.page_mapping = {
            1: "1/12",  # Page 1 should show 1/12
            2: "5/12",  # Page 2 should show 5/12
            3: "7/12",  # Page 3 should show 7/12
            4: "8/12",  # Page 4 should show 8/12
            5: "10/12", # Page 5 should show 10/12
        }
        
    def header(self):
        """
        Add a Plug2Drive logo as header in each page of the PDF
        Remainder: here is a local path of the logo - Try to use a safe
        link that can redirect to logo image
        """
        self.image('/Users/.../Logo Plug2drive.jpg',
              15,14,17)
        

    def footer(self):
        """Custom footer to show page numbers as 5/12, 7/12, etc."""
        self.set_y(-15)  # Position footer 15mm from the bottom
        self.set_font('helvetica', '', 8)  # Italic font for footer
        current_page = self.page_no()
        
        # Use page_mapping to assign the correct page number
        page_text = self.page_mapping.get(current_page, f"{current_page}/{self.total_pages}")

        # Center align the footer
        self.cell(0, 10, page_text, align='R')

## Page 1: Title + Info Table

In [11]:
def page_1_pdf(pdf,folder_name: str, excel_data: dict, images: dict) -> str:
    """
    PDF page that recreates template first page using data collected 
    in Info sheet and image from "Façade" folder
    Page 1 element:
    - Table which contains site "Info" data
    - A photo of the site's façade retrieve from "Façade" subfolder
    - Plug2Drive stamp
    The function will raise an error message if folder_name or excel_data
    are missing.
    
    :param pdf:
    :type pdf:
    :param folder_name: Site's name (= adress)
    :type folder_name: str
    :param excel_data: Data retrieve from excel file
    :type excel_data: dict
    :param images: Images collected site folder and its subfolders
    :type images: dict
    :raiseValueError: if folder_name or excel_data are missing.
    
    :Example:
    >>> page_1_pdf(pdf,folder_name, excel_data, images)
    page_1_pdf = PDF page
    
    """

    pdf.add_page()
    
    doc_w = pdf.w  # Document width for centering


    ####### Rapport : N°XXX Title #######
    title = f"Rapport : {folder_name}"
    pdf.set_font('helvetica', 'B', 19)

    title_w = pdf.get_string_width(title) + 3
    pdf.set_x((doc_w - title_w) / 2)
    pdf.cell(title_w, 55, title, border=False, new_x=XPos.LMARGIN, new_y=YPos.NEXT, 
             align='C')
    

    ######## Réglage des organes... #####
    title1 = "Réglage des organes d'équilibrage d'une installation de chauffage à eau"
    pdf.set_font('helvetica', 'B', 14)
    title1_w = pdf.get_string_width(title1) + 3
    pdf.set_x((doc_w - title1_w) / 2)
    pdf.cell(title1_w, -38, title1, border=False, new_x=XPos.LMARGIN, new_y=YPos.NEXT, 
             align='C')

    ######## chaude ######
    title2 = "chaude"
    pdf.set_font('helvetica', 'B', 14)
    title2_w = pdf.get_string_width(title2) + 3
    pdf.set_x((doc_w - title2_w) / 2)
    pdf.cell(title2_w, 49, title2, border=False, new_x=XPos.LMARGIN, new_y=YPos.NEXT, 
             align='C')

    ######## Adress ######
    address = excel_data["Équilibrage_Site_Logement"]["Site_Info"]
    pdf.set_font('helvetica', '', 13)
    address_w = pdf.get_string_width(address) + 3
    pdf.set_x((doc_w - address_w) / 2)
    pdf.cell(address_w, -25, address, border=False, new_x=XPos.LMARGIN, new_y=YPos.NEXT, 
             align='C')

    ###### TABLE (INFO DATA) FORMAT ######
    pdf.set_font('helvetica', '', 10)
    info_dict = excel_data['Info']
    col_widths = [60, 100]
    table_width = sum(col_widths)
    start_x = (doc_w - table_width) / 2
    manual_y_position = 88
    pdf.set_y(manual_y_position)
    
    ######## INFO ELEMENT PLACEMENT INSIDE THE TABLE ABOVE ######

    for key, value in info_dict.items():
        pdf.set_x(start_x)
        pdf.cell(col_widths[0], 8, key, border=1, align='C')
        pdf.cell(col_widths[1], 8, str(value), border=1, new_x=XPos.LMARGIN, new_y=YPos.NEXT, 
                 align='C')

    ###### FAÇADE PHOTO BELOW TABLE ######
    pdf.ln(10)
    image_width = 50
    image_x_position = (doc_w - image_width) / 2
    image_y_position = pdf.get_y()
    
    if images and "Façade" in images:
        pdf.set_font('helvetica', 'B', 12)
        image_path = images["Façade"][0]
        pdf.image(image_path, x=image_x_position, y=image_y_position, w=image_width)
        pdf.ln(60)  # Space for the next section
        
    else:
        pdf.set_font('helvetica', '', 12)  
        missing_message = "Façade Photos missing"
        message_façade_width = pdf.get_string_width(missing_message)
        message_façade_x_position = (doc_w - message_façade_width) / 2
        pdf.set_xy(message_façade_x_position, pdf.get_y() + 65)  # Set position for the message
        pdf.cell(message_façade_width, 10, missing_message, align='C')  # Center the message    
        
    ### CACHET SIGNATURE (STAMP) ###
    
    # Cachet_signature TABLE FORMAT
    cachet_table_y_position = image_y_position + 50  # Adjust based on the image height
    pdf.set_y(cachet_table_y_position)
    pdf.set_font('helvetica', '', 7)

    # Define table structure TABLE FORMAT
    cachet_col_widths = [55, 55] 
    row1_height = 3
    row2_height = 25
    cachet_table_width = sum(cachet_col_widths)
    cachet_start_x = (doc_w - cachet_table_width) / 2

    # ADD FIRST ROW ELEMENT (Intervenant & Client)
    pdf.set_x(cachet_start_x)
    pdf.cell(cachet_col_widths[0], row1_height, "Intervenant", border=1, align='C')
    pdf.cell(cachet_col_widths[1], row1_height, "Client", border=1, new_x=XPos.LMARGIN, new_y=YPos.NEXT, 
             align='C')

    # ADD SECOND ROW ELEMENT (Plug2Drive & Client Stamp)
    pdf.set_x(cachet_start_x)
    # Empty cell for "Cachet"
    pdf.cell(cachet_col_widths[0], row2_height, "", border=1)
    # Empty cell for "Signature"
    pdf.cell(cachet_col_widths[1], row2_height, "", border=1, new_x=XPos.LMARGIN, new_y=YPos.NEXT)
    
    ########## ADDING STAMP TO "CACHET" CELL (TEST WITH AN IMAGE) ##########
    
    
    # Add the second (larger) row with an image in the "Cachet" cell
    pdf.set_x(cachet_start_x)
    cachet_x = pdf.get_x()  # Get the current X position for the "Cachet" cell
    cachet_y = pdf.get_y()  # Get the current Y position for the "Cachet" cell

    # Define the dimensions of the "Cachet" cell
    cachet_width = cachet_col_widths[0]
    cachet_height = row2_height

    # Add the image inside the "Cachet" cell
    cachet_image_path = "/Users/.../Cachet Plug2Drive.png"  # Path to the image file # Add link 
    image_width = cachet_width - 2  # Slightly smaller than the cell width for padding
    image_height = cachet_height - 3  # Slightly smaller than the cell height for padding

    # Calculate the top-left position for centering the image inside the cell
    image_x = cachet_x + (cachet_width - image_width ) / 2  
    image_y = cachet_y + (cachet_height - image_height) / 2      

    # Insert the image
    pdf.image(cachet_image_path, x=image_x, y=200, w=image_width, h=image_height)

## Page 5: Installation et Réseaux

In [44]:
def page_5_pdf(pdf, images: dict) -> str:
    """
    PDF page that recreates template fifth page using
    photos collected from "Chaufferie" and "Pompes" folder
    if exists
    Page 5 element:
    - Photo of the site's boiler retrieved from "Chaufferie" subfolder
    - Photo of the site's pump retrieved from "Pompes" subfolder
    The function will raise an error message if folder_name or excel_data
    are missing.
    
    :param pdf:
    :type pdf:
    :param excel_data: Data retrieve from excel file
    :type excel_data: dict
    :param images: Images collected site folder and its subfolders
    :type images: dict
    :raiseValueError: if folder_name or excel_data are missing.
    
    :Example:
    >>> page_1_pdf(pdf,folder_name, excel_data, images)
    page_1_pdf = PDF page
    
    """

    pdf.add_page()
    
    doc_w = pdf.w # Document width for centering
    
    ####### 4.Installation et Réseaux #######
    title_1 = "4. Installation et Réseaux"
    pdf.set_font('helvetica', 'B', 12)

    title_1_width = pdf.get_string_width(title_1) + 3
    pdf.set_x(29)
    pdf.cell(title_1_width, 58, title_1, border=False, new_x=XPos.RIGHT, new_y=YPos.TOP, 
             align='L')
    
    ####### 4.1 Chaufferie #######
    title_2 = "4.1 Chaufferie"
    pdf.set_font('helvetica', 'B', 12)

    title_2_width = pdf.get_string_width(title_2) + 3
    pdf.set_x(29)
    pdf.cell(title_2_width, 79, title_2, border=False, new_x=XPos.RIGHT, new_y=YPos.TOP, 
             align='L')
    
    ####### PARTIE CHAUFFERIE - SENTENCE + PHOTO #######
    
    ####### Chaufferie sentence #######
    title_3 = "La résidence est chauffée via une chaudière DE DIETRICH"
    pdf.set_font('helvetica', '', 11)

    title_3_width = pdf.get_string_width(title_3) + 3
    pdf.set_x(29)
    pdf.cell(title_3_width, 109, title_3, border=False, new_x=XPos.RIGHT, new_y=YPos.TOP, 
             align='L')
    
    # Initialize image_y_position in case no images are available
    image_x_position = (doc_w - 50) / 2  # Centered position for "chaufferie" AND "pompe" images
    image_width = 50  # Width for "chaufferie" AND "pompe" images
    image_height = 60  # Height for "chaufferie" AND "pompe" images
    
    image_1_y_position = pdf.get_y() + 65  # Image 1 - Y position if image is present
    
    # Add Image 1 (First from Chaufferie)
    
    if "Chaufferie" in images and images["Chaufferie"]:
        image_chaufferie_path = images["Chaufferie"][0]
        
        ############ Image Rotation ############
        # Open the image and rotate it
        with Image.open(image_chaufferie_path) as img:
            rotated_image_chaufferie_path = "rotated_image_1.jpg" # Need to modify path and include Chaufferie folder path
            rotated_img = img.rotate(270, expand=True)  # Rotate 90 degrees clockwise, expand to fit
            rotated_img.save(rotated_image_chaufferie_path)

        pdf.image(rotated_image_chaufferie_path, x=image_x_position, y=image_1_y_position, w=image_width)
    
    else:
      # Display "Chaufferie Photos missing" if no images available
        pdf.set_font('helvetica', 'I', 12)  # Italic font for the message
        missing_message = "Chaufferie Photos missing"
        message_width = pdf.get_string_width(missing_message)
        message_x_position = (doc_w - message_width) / 2  # Center the text
        pdf.set_xy(message_x_position, pdf.get_y() + 65)  # Set position for the message
        pdf.cell(message_width, 10, missing_message, align='C')  # Center the message
            
        
    # Adjust for next content
    image_height = 60  # Adjust image height if necessary
    pdf.set_y(image_1_y_position + image_height + 10)
    
    
    ####### PARTIE POMPE - SENTENCE + PHOTO #######
    
    
    ####### Pompe sentence #######
    title_4 = "Il a été installée une pompe SALMSON, afin de réguler la circulation de l'eau et du chauffage"
    pdf.set_font('helvetica', '', 11)

    title_4_width = pdf.get_string_width(title_4) + 3
    pdf.set_x(29)
    pdf.cell(title_4_width, 20, title_4)
    
    title_5 = "de la résidence."
    pdf.set_font('helvetica', '', 11)

    title_5_width = pdf.get_string_width(title_5) + 3
    pdf.set_x(29)
    pdf.cell(title_5_width, 33, title_5)
    
    image_2_y_position = pdf.get_y() + 25

    # Add Image 2 (First from Pompes folder)
    
    if "Pompes" in images and images["Pompes"]:
        image_pompe_path = images["Pompes"][0]
        
        ############ Image Rotation ############
        # Open the image and rotate it
        with Image.open(image_pompe_path) as img:
            rotated_image_pompe_path = "rotated_image_2.jpg" # Need to modify path and include Pompes folder path
            rotated_img = img.rotate(270, expand=True)  # Rotate 90 degrees clockwise, expand to fit
            rotated_img.save(rotated_image_pompe_path)
        
        pdf.image(rotated_image_pompe_path, x=image_x_position, y=image_2_y_position, w=image_width)

    else:
      # Display "Chaufferie Photos missing" if no images available
        pdf.set_font('helvetica', 'I', 12)  # Italic font for the message
        missing_message_pompes = "Photo du dossier:'Pompes' manquantes"
        message_pompes_width = pdf.get_string_width(missing_message_pompes)
        message_pompes_x_position = (doc_w - message_pompes_width) / 2  # Center the text
        pdf.set_xy(message_pompes_x_position, pdf.get_y() + 65)  # Set position for the message
        pdf.cell(message_pompes_width, 10, missing_message_pompes, align='C')  # Center the message

    # Ensure that you add a line break at the end of the second page
    pdf.ln(20)

## Page 7: Tableau Relevé Equilibrage

In [38]:
def page_7_pdf(pdf, excel_data: dict) -> str:
    """
    PDF page that recreates template page n°7 using
    using data collected in 'Équilibrage' sheet from the Excel
    file (if exists).
    Page 7 element:
    - Table which contains site "Équilibrage" data
    The function will raise an error message if data from excel_data
    are missing.
    
    :param pdf:
    :type pdf:
    :param excel_data: Data retrieve from excel file
    :type excel_data: dict
    :raiseValueError: if excel_data are missing.
    
    :Example:
    >>> page_7_pdf(pdf, excel_data)
    page_7_pdf = PDF page
    """
    
    ####################################################################################
    
    pdf.add_page()
    
    pdf.set_font("helvetica", "B", 13)
    
    ####### TABLEAU - PARTIE DYNAMIC WIDTH FORMAT #######

    ####### DYNAMIC TABLE WIDTH #######
    page_margin = 16 #18  # Left & right margin
    usable_width = pdf.w - 2 * page_margin  # Page width minus margins

    # Extract headers and data
    data_as_dict = excel_data["Équilibrage_Table_Data"] 
    headers = list(data_as_dict.keys()) # Retrieve keys from dict to use it as headers
    rows = list(zip(*data_as_dict.values()))  # Transpose values to get rows

    # Dynamically calculate column widths
    def calculate_cell_width(header, column_values, wrap_width=12):
        """
        Calculate cell_width based on length of headers and rows string in order to
        create a dynamic table so that each string elements won't overflow other columns/rows.
        
        :param header: Represents excel_data["Équilibrage_Table_Data"] keys element
        :type header: list
        :param column_values: Represents excel_data["Équilibrage_Table_Data"] values element
        :type column_values: list
        :param wrap_width=12: Default value for cell width
        """
        all_texts = [header] + [str(value) for value in column_values]  
        max_width = max(pdf.get_string_width(line) for text in all_texts for line in textwrap.wrap(text, 
                                                                                                   wrap_width))
        return max_width + 4  # Add padding

    column_widths = [calculate_cell_width(headers[i], [row[i] for row in rows]) for i in range(len(headers))]
    table_width = sum(column_widths)  # Calculate total table width

    # Ensure table width does not exceed usable width
    if table_width > usable_width:
        scale_factor = usable_width / table_width
        column_widths = [w * scale_factor for w in column_widths]
        table_width = usable_width
        
    ####################################################################################

    ####### "TABLEAU DE RELEVE D'EQUILIBRAGE" CENTERED TITLE (WITH BORDER) #######
    
    title = "TABLEAU DE RELEVE D'EQUILIBRAGE"
    title_width = table_width  # Use table width for title to match with Site & Logement info
    title_x = page_margin + (usable_width - table_width) / 2  # Center title within the usable width
    title_y = 50  # Fixed top position

    # Draw title cell with dynamic width matching table
    pdf.set_xy(title_x, title_y)
    pdf.cell(w=title_width, h=11.5, text=title, border=1, align='C', new_x=XPos.LMARGIN, 
             new_y=YPos.NEXT)

    # Add spacing for next section
    next_y = title_y + 11.5 + 7
    
    ####################################################################################
    
    ####### SITE INFO & LOGEMENT INFO (DYNAMIC WIDTH) #######
    pdf.set_font('helvetica', 'B', 9)

    # Get dynamic content
    site_info = excel_data["Équilibrage_Site_Logement"]["Site_Info"]
    logement_info = excel_data["Équilibrage_Site_Logement"]["Logement_Info"]

    # Ensure total width matches the table width
    site_width = table_width * 0.7  # site_width must be larger than logement_width
    logement_width = table_width * 0.3

    # Draw "Site Info" cell
    pdf.set_xy(title_x, next_y)  # Align with the table
    pdf.cell(w=site_width, h=10, border=1, text=site_info, align='L')

    # Draw "Logement Info" cell
    pdf.set_xy(title_x + site_width, next_y)
    pdf.cell(w=logement_width, h=10, border=1, text=logement_info, align='L')

    # Update Y position for the table
    next_y += 10 + 8 #5  # Move down after this section
    
    ####################################################################################

    ####### DRAW THE TABLE BELOW (WITH BORDERS) #######
    pdf.set_font("helvetica", "B", size=11)

    # Define cell dimensions
    header_cell_height = 17   # Height of header (dict.keys) cells
    data_cell_height = 8      # Height of data (dict.values) cells
    padding = 2               # Padding inside the cell
    
    # Function to create a custom cell with borders
    def create_custom_cell(x, y, text, cell_width, cell_height, is_bold, border=1):
        """
        Create a custom format cell based on the text (param) format.
        
        :param x: cell x-position (horizontal)
        :type x: int or float
        :param y: cell y-position (vertical)
        :type y: int or float
        :param cell_width: cell width
        :type cell_width: int or float
        :param cell_height: cell height
        :type cell_height: int or float
        :param is_bold: determines if text is bold
        :type is_bold: boolean
        :param border: determines if the cell has border
        :type border: boolean
        """
        pdf.set_font("helvetica", "B" if is_bold else "", size=9.5)
        wrapped_lines = textwrap.wrap(str(text), width=12)
        text_block_height = 5 * len(wrapped_lines)
        vertical_offset = (cell_height - text_block_height) / 2

        # Draw the border
        pdf.set_xy(x, y)
        pdf.cell(w=cell_width, h=cell_height, border=border, new_x=XPos.RIGHT, 
                 new_y=YPos.TOP)

        # Add the text, if any
        for i, line in enumerate(wrapped_lines):
            text_width = pdf.get_string_width(line)
            horizontal_offset = (cell_width - text_width) / 2
            pdf.set_xy(x + horizontal_offset, y + vertical_offset + i * 5)
            pdf.cell(w=text_width, h=5, text=line, border=0, align='C')

    ####### DRAW HEADER ROW (WITH BORDERS) #######
    current_x = title_x
    header_y = next_y

    for col_index, header in enumerate(headers):
        create_custom_cell(current_x, header_y, header, column_widths[col_index], header_cell_height, is_bold=True,
                           border=1)
        current_x += column_widths[col_index]

    # Move to the next row for plain row and data
    next_y = header_y + header_cell_height
    
    ####################################################################################

    ####### PLAIN ROW (OUTER BORDER SPANNING THE WHOLE TABLE WIDTH) #######
    plain_row_y = next_y  # Plain row just after the header row
    plain_row_height = 10  # Height of the plain row cell

    # Draw the plain cell with an outer border spanning the whole table width
    pdf.set_xy(title_x, plain_row_y)
    pdf.cell(w=table_width, h=plain_row_height, border=1, new_x=XPos.RIGHT, new_y=YPos.TOP)  # Outer border only

    # Update Y position for data rows after the plain row
    next_y = plain_row_y + plain_row_height  # Adjust Y for data rows after the plain row
    
    ####################################################################################

    ####### DRAW DATA ROWS (WITH BORDERS) #######
    for row_index, row in enumerate(rows):
        data_y = next_y + (row_index * data_cell_height)  # Adjust Y position for each row
        current_x = title_x  # Reset to first column

        for col_index, cell_text in enumerate(row):
            create_custom_cell(
                current_x, 
                data_y, 
                cell_text, 
                column_widths[col_index], 
                data_cell_height, 
                is_bold=False,
                border=1  # Ensure each cell has a border
            )
            current_x += column_widths[col_index]  # Move to next column
            
    ####################################################################################
            
    ####### PLAIN ROW (OUTER BORDER SPANNING THE WHOLE TABLE WIDTH) #######
    footer_y = data_y + data_cell_height #+ 5
    pdf.set_xy(title_x, footer_y)
    pdf.set_font("helvetica", "", 9)
    pdf.multi_cell(w=table_width, h=10, border=0, 
                   text="Observations: C = Conforme, NC = Non Conforme, NE = Non exécuté", align='L')

## Page 8: Tableau Enregistrement Températures

In [36]:
def page_8_pdf(pdf,excel_data: dict) -> str:
    """
    PDF page that recreates template page n°8 using
    using data collected in 'Températures' sheet from the Excel
    file (if exists).
    Page 8 element:
    - Table which contains site "Températures" data
    The function will raise an error message if data from excel_data
    are missing.
    
    :param pdf:
    :type pdf:
    :param excel_data: Data retrieve from excel file
    :type excel_data: dict
    :raiseValueError: if excel_data are missing.
    
    :Example:
    >>> page_8_pdf(pdf, excel_data)
    page_8_pdf = PDF page
    
    """
    
    ############ PAGE TEMPERATURE #############
    pdf.add_page()

    # Add Tableau Title
    pdf.set_font("helvetica", "B", 13)
    pdf.set_xy(15, 37)
    pdf.cell(w=180, h=11.5, text="TABLEAU D'ENREGISTREMENT DES TEMPÉRATURES", border=1, 
             align='C', new_x=XPos.LMARGIN, new_y=YPos.NEXT)

    # Add 5mm spacing
    next_y = 37 + 11.5 + 5
    
    ####################################################################################
    
    ####### SITE + LOGEMENT INFOS #######
    #Redefine site_info and logement_info with element extracted
    site_info = excel_data["Équilibrage_Site_Logement"]["Site_Info"]
    logement_info = excel_data["Équilibrage_Site_Logement"]["Logement_Info"]

    # Site + Nbr Logement
    pdf.set_font('helvetica', 'B', 9)
    pdf.set_xy(15, next_y)
    pdf.multi_cell(w=100, h=10, border=1, text= site_info)
    pdf.set_xy(115, next_y)
    pdf.multi_cell(w=80, h=10, border=1, text= logement_info)
    
    ####################################################################################

    # Add 5mm spacing after the second row
    next_y += 10 + 5

    # Set table font
    pdf.set_font("helvetica", "B", size=11)

    # Define constants
    header_cell_height = 20    # Height for header cells
    data_cell_height = 10      # Height for data cells
    plain_row_height = 10.5    # Height for plain row (spacing)
    padding = 4                # Padding inside the cell
    line_height = 6            # Line height for text
    

    # Function to calculate the width of a cell dynamically
    def calculate_cell_width(header, column_values):
        """
        Calculate cell width base on header string length and its
        column value to adjust columns widths dynamically so that
        table avoid overflowing.
        
        :param headers: The headers that will displayed in the table
        :type headers: list
        :param column_values: The data values that will be displayed in the table
        :type column_values: list
        """
        all_texts = [header] + [str(value) for value in column_values]
        max_width = max(pdf.get_string_width(text) for text in all_texts)
        return max_width + 2 * padding

    # Function to create a centered cell with optional borders
    def create_custom_cell(x, y, text, cell_width, cell_height, is_bold=False, border=1):
        """
        Create a custom format cell based on the text (param) format.
        
        :param x: cell x-position (horizontal)
        :type x: int or float
        :param y: cell y-position (vertical)
        :type y: int or float
        :param cell_width: cell width
        :type cell_width: int or float
        :param cell_height: cell height
        :type cell_height: int or float
        :param is_bold: determines if text is bold
        :type is_bold: boolean
        :param border: determines if the cell has border
        :type border: boolean
        """

        pdf.set_font("helvetica", "B" if is_bold else "", size=11)
        text_width = pdf.get_string_width(str(text))
        horizontal_offset = (cell_width - text_width) / 2
        vertical_offset = (cell_height - line_height) / 2

        # Draw the border
        pdf.set_xy(x, y)
        pdf.cell(w=cell_width, h=cell_height, border=border, new_x=XPos.RIGHT, new_y=YPos.TOP)

        # Place the text
        pdf.set_xy(x + horizontal_offset, y + vertical_offset)
        pdf.cell(w=text_width, h=line_height, text=str(text), border=0, align='C')
        
    
    temp_dict = excel_data["Températures"] 

    # Headers and rows
    headers = list(temp_dict.keys())  # Headers based on the keys of the dictionary
    rows = list(zip(*temp_dict.values()))  # Transpose data to get rows

    # Calculate column widths based on totla number of headers element
    column_widths = [
        calculate_cell_width(headers[col_index], [row[col_index] for row in rows])
        for col_index in range(len(headers))
    ]

    # Calculate total table width and starting X position for centering
    table_width = sum(column_widths)
    page_width = pdf.w  # Page width
    start_x = (page_width - table_width) / 2

    ####### DRAW HEADER ROW #######
    header_y = next_y
    current_x = start_x
    column_positions = []

    for col_index, header in enumerate(headers):
        column_positions.append(current_x)
        create_custom_cell(current_x, header_y, header, column_widths[col_index], header_cell_height, 
                           is_bold=True, border=1)
        current_x += column_widths[col_index]

    ####### DRAW PLAIN ROW BELOW THE HEADER #######
    plain_row_y = header_y + header_cell_height
    pdf.set_xy(start_x, plain_row_y)
    pdf.cell(w=table_width, h=plain_row_height, border=1, new_x=XPos.RIGHT, new_y=YPos.TOP)

    ####### DRAW DATA ROWS #######
    for row_index, row in enumerate(rows):
        data_y = plain_row_y + plain_row_height + (row_index * data_cell_height)
        for col_index, cell_text in enumerate(row):
            create_custom_cell(
                column_positions[col_index], 
                data_y, 
                cell_text, 
                column_widths[col_index], 
                data_cell_height, 
                is_bold=False, 
                border=1
            )
                    
    ####################################################################################
                
                
    # Calculate Y position after last data row
    last_data_y = plain_row_y + plain_row_height + (len(rows) * data_cell_height)

    # New two-row table below the current table
    new_table_headers = ["Signature et cachet Entreprise du client"]
    new_table_data = [""]  # Second row will be empty

    new_table_column_width = calculate_cell_width(new_table_headers[0], new_table_data)

    # Adjust the Y position to move the header closer to the top
    new_table_y = last_data_y + 5  # Keep the Y position closer to the data

    # Function to adjust the vertical offset only for the last table header
    def create_last_table_header(x, y, text, cell_width, cell_height, is_bold=False, border=1):
        """
        Create a custom table for client stamp.
        
        :param x: cell x-position (horizontal)
        :type x: int or float
        :param y: cell y-position (vertical)
        :type y: int or float
        :param cell_width: cell width
        :type cell_width: int or float
        :param cell_height: cell height
        :type cell_height: int or float
        :param is_bold: determines if text is bold
        :type is_bold: boolean
        :param border: determines if the cell has border
        :type border: boolean
        """
        
        pdf.set_font("helvetica", "B" if is_bold else "", size=11)
        text_width = pdf.get_string_width(str(text))
        horizontal_offset = (cell_width - text_width) / 2
        
        # Adjust vertical offset to move the text closer to the top for the last table
        vertical_offset = (cell_height - line_height) / 4  # Move text closer to the top

        # Set text color to blue (RGB values for blue: 0, 0, 255)
        pdf.set_text_color(0, 0, 255)  # Bleu code couleur

        # Draw the border
        pdf.set_xy(x, y)
        pdf.cell(w=cell_width, h=cell_height, border=border, new_x=XPos.RIGHT, new_y=YPos.TOP)

        # Place the text
        pdf.set_xy(x + horizontal_offset, y + vertical_offset)
        pdf.cell(w=text_width, h=line_height, text=str(text), border=0, align='C')

        # Reset text color back to black after the header
        pdf.set_text_color(0, 0, 0)  # Black color for other text


    # Draw header row for the new table (with border)
    current_x = (page_width - new_table_column_width) / 2  # Ensure the table is centered
    create_last_table_header(current_x, new_table_y, new_table_headers[0], new_table_column_width, 
                             header_cell_height, is_bold=True, border="LRT")

    # Draw first data row (with no top border)
    first_data_row_y = new_table_y + header_cell_height  # Position for the first data row
    create_custom_cell(current_x, first_data_row_y, new_table_data[0], new_table_column_width, 
                       data_cell_height, is_bold=False, border="LR")

    # Draw second row (empty) with border
    second_data_row_y = first_data_row_y + data_cell_height
    create_custom_cell(current_x, second_data_row_y, "", new_table_column_width, 
                       data_cell_height, is_bold=False, border="LRB")

## Page 10: Vannes Photos

In [37]:
def page_10_pdf(pdf,folder_name: str,images: dict) -> str:
    """
    PDF page that recreates template page n°10 using
    photos found in "Vannes" subfolder (if exists).
    The function will raise an error message if no photos
    are found.
    
    :param pdf:
    :type pdf:
    :param folder_name: Site's name (= adress)
    :type folder_name: str
    :param images: Images collected site folder and its subfolders
    :type images: dict
    :raiseValueError: if photos are missing.
    
    :Example:
    >>> page_8_pdf(pdf, excel_data)
    page_8_pdf = PDF page
    """
    pdf.add_page()
    
    doc_w = pdf.w  # Document width for centering
    image_width = 50
    image_x_position = (doc_w - image_width) / 2

    if "Vannes" in images and images["Vannes"]:
        for i, image_vannes_path in enumerate(images["Vannes"][:3]):  # Limit to 3 images
            image_y_position = pdf.get_y() + 30 + (i * 40)  # Adjust spacing dynamically
            pdf.image(image_vannes_path, x=image_x_position, y=image_y_position, w=image_width)
    else:
        pdf.set_font("helvetica", "", 12)
        
        # Calculate the width of the text to be centered
        text = f"Photos Vannes introuvables dans le dossier {folder_name}"
        text_width = pdf.get_string_width(text)
        
        # Calculate the new x position to center the text based on image_x_position
        text_x_position = image_x_position + (image_width - text_width) / 2
        
        # Add the centered text at the calculated position
        pdf.set_xy(text_x_position, pdf.get_y() + 30)  # Set x position and y position for the cell
        pdf.cell(text_width, 10, text, align="C")  # Center align the text

# Combining the Output Report with the Template

In [16]:
def merging_reports(folder, template_pdf_path, pdf_output_path):
    """
    Merge page from previous function created above 
    with template file located in each site folder.
    "PLUG2DRIVE - Rapport équilibrage - 6 10 rue Fizeau 92150 Suresnes.pdf"
    If one of the parameter is invalid, the function raises a ValueError.
    
    :param folder: Site's Path
    :type folder: Path
    :param template_pdf_path: PDF Template Report Path
    :type template_pdf_path: Path
    :param pdf_output_path: PDF of page create previously (Page 1/5/7/8/10)
    :type pdf_output_path: Path
    :raises ValueError: if folder, template_pdf_path or pdf_out_path is
    missing.
    """
    try:
                
        ########### MERGING OUTPUT_PATH WITH TEMPLATE ###########
            
                
        merged_pdf_name = f"{os.path.basename(folder)}_merged.pdf"
        merged_pdf_path = os.path.join(folder, merged_pdf_name)
            
        template_reader = PdfReader(template_pdf_path)
        generated_reader = PdfReader(pdf_output_path)

        # Initialize the writer
        writer = PdfWriter()

        # Add pages in the desired order
        writer.add_page(generated_reader.pages[0])  # Generated PDF - Page 1
        writer.add_page(template_reader.pages[1])  # Template PDF - Page 3
        writer.add_page(template_reader.pages[2])  # Template PDF - Page 4
        writer.add_page(template_reader.pages[3])  # Template PDF - Page 6
        writer.add_page(generated_reader.pages[1])  # Generated PDF - Page 5
        writer.add_page(template_reader.pages[5])
        writer.add_page(generated_reader.pages[2])  # Generated PDF - Page 7
        writer.add_page(generated_reader.pages[3])  # Generated PDF - Page 8                   
        writer.add_page(template_reader.pages[8])  # Template PDF - Page 9
        writer.add_page(generated_reader.pages[4])  # Generated PDF - Page 10
        writer.add_page(template_reader.pages[10])
        writer.add_page(template_reader.pages[11])

        # Save the merged PDF
        with open(merged_pdf_path, "wb") as merged_file:
            writer.write(merged_file)

        print(f"\n Rapport après fusion créé : {merged_pdf_path}\n")
    except Exception as e:
        print(f"Erreur rencontrée lors de la fusion des PDF pour dossier {folder}: {e}")   

# Execute All Functions Together

In [47]:
# Function 3: Process all folders and extract data
def process_folders_and_files() -> str:
    """
    Process all folders using the get_site_folder() function, 
    which retrieves all matching folders. Then, generate a separate 
    PDF only if both Excel data and images exist. Finally, merge 
    the separate PDF with the required pages from the template PDF 
    to generate the final automated PDF.
    The function return information on site's report which has been
    generated.
    It raises an error message if folder, excel file or image folder 
    is missing
    """
    
    folders = get_site_folder()  # Retrieve all folders
    
    for folder in folders:
        folder_name = folder.name
        excel_data = None
        images = extract_image(folder)  # Extract images first
        template_pdf_path = os.path.join(folder, "PLUG2DRIVE - Rapport équilibrage - 6 10 rue Fizeau 92150 Suresnes.pdf")
        
        # Check if the template PDF exists
        template_present = os.path.exists(template_pdf_path)
        total_pages = 12 #parameter needed in CustomPDF class
        

        # Look for the Excel file
        for file in folder.iterdir():
            if file.name.startswith("donnees_clients") and file.suffix == ".xlsx":
                excel_data = extract_excel_data(file)
                break  # Stop searching after finding the first valid Excel file

        # **Check if both excel_data and images exist before generating PDF**
        if excel_data and images and template_present:
            pdf = CustomPDF("P", "mm", "A4", total_pages)
            pdf.alias_nb_pages()

            # Generate PDF pages
            page_1_pdf(pdf, folder_name, excel_data, images)
            page_5_pdf(pdf, images)
            page_7_pdf(pdf, excel_data)
            page_8_pdf(pdf, excel_data)            
            page_10_pdf(pdf,folder_name, images)

            # Save the PDF separately for each folder
            pdf_output_path = f"Rapport_{folder_name}.pdf"
            pdf.output(pdf_output_path)
            
            merging_reports(folder, template_pdf_path, pdf_output_path)

            
            print(f"Rapport créé : {pdf_output_path}\n")
        else:
            print(f"Dossier '{folder_name}' non étudié (Fichier Excel (donnees_clients), Images ou Template manquant.)")

# Main block: Process folders and extract data
if __name__ == "__main__":
    process_folders_and_files()